In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,29.86,93,100,19.57,heavy snow
1,1,Tuktoyaktuk,CA,69.4541,-133.0374,51.80,93,100,6.91,overcast clouds
2,2,Vaini,TO,-21.2000,-175.2000,74.01,75,5,9.33,clear sky
3,3,Albany,US,42.6001,-73.9662,88.68,64,8,1.01,clear sky
4,4,Yellowknife,CA,62.4560,-114.3525,63.12,55,20,5.99,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Albany,US,42.6001,-73.9662,88.68,64,8,1.01,clear sky
11,11,Kapaa,US,22.0752,-159.3190,80.19,81,100,19.57,light rain
14,14,Nikolskoye,RU,59.7035,30.7861,82.87,50,0,4.09,clear sky
15,15,Noyon,FR,49.5833,3.0000,81.25,51,94,9.37,overcast clouds
20,20,Nara,JP,34.6851,135.8049,79.50,69,87,0.69,overcast clouds
21,21,Fort-Shevchenko,KZ,44.5086,50.2630,77.36,68,63,22.19,broken clouds
22,22,Hami,CN,42.8000,93.4500,78.66,21,0,2.17,clear sky
26,26,Karpathos,GR,35.5070,27.2132,76.96,69,0,33.38,clear sky
27,27,Jamestown,US,42.0970,-79.2353,79.02,74,100,14.97,overcast clouds
28,28,Hasaki,JP,35.7333,140.8333,79.29,91,100,13.60,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                285
City                   285
Country                283
Lat                    285
Lng                    285
Max Temp               285
Humidity               285
Cloudiness             285
Wind Speed             285
Current Description    285
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna(axis=0, how='any', inplace=False)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Albany,US,88.68,clear sky,42.6001,-73.9662,
11,Kapaa,US,80.19,light rain,22.0752,-159.3190,
14,Nikolskoye,RU,82.87,clear sky,59.7035,30.7861,
15,Noyon,FR,81.25,overcast clouds,49.5833,3.0000,
20,Nara,JP,79.50,overcast clouds,34.6851,135.8049,
21,Fort-Shevchenko,KZ,77.36,broken clouds,44.5086,50.2630,
22,Hami,CN,78.66,clear sky,42.8000,93.4500,
26,Karpathos,GR,76.96,clear sky,35.5070,27.2132,
27,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,
28,Hasaki,JP,79.29,overcast clouds,35.7333,140.8333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Albany,US,88.68,clear sky,42.6001,-73.9662,
11,Kapaa,US,80.19,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Nikolskoye,RU,82.87,clear sky,59.7035,30.7861,Tourist House - Sablino
15,Noyon,FR,81.25,overcast clouds,49.5833,3.0000,Cit'Hotel le Cèdre
20,Nara,JP,79.50,overcast clouds,34.6851,135.8049,Guesthouse Nara Komachi
21,Fort-Shevchenko,KZ,77.36,broken clouds,44.5086,50.2630,Oasis hotel Bautino
22,Hami,CN,78.66,clear sky,42.8000,93.4500,Laoyutang Resort
26,Karpathos,GR,76.96,clear sky,35.5070,27.2132,Alex
27,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
28,Hasaki,JP,79.29,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel


In [10]:
hotel_df.count()

City                   283
Country                283
Max Temp               283
Current Description    283
Lat                    283
Lng                    283
Hotel Name             283
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Kapaa,US,80.19,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Nikolskoye,RU,82.87,clear sky,59.7035,30.7861,Tourist House - Sablino
15,Noyon,FR,81.25,overcast clouds,49.5833,3.0000,Cit'Hotel le Cèdre
20,Nara,JP,79.50,overcast clouds,34.6851,135.8049,Guesthouse Nara Komachi
21,Fort-Shevchenko,KZ,77.36,broken clouds,44.5086,50.2630,Oasis hotel Bautino
22,Hami,CN,78.66,clear sky,42.8000,93.4500,Laoyutang Resort
26,Karpathos,GR,76.96,clear sky,35.5070,27.2132,Alex
27,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
28,Hasaki,JP,79.29,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
30,Alexandria,EG,80.01,clear sky,31.2156,29.9553,Royal Crown Hotel


In [12]:
clean_hotel_df.count()

City                   266
Country                266
Max Temp               266
Current Description    266
Lat                    266
Lng                    266
Hotel Name             266
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Kapaa,US,80.19,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Nikolskoye,RU,82.87,clear sky,59.7035,30.7861,Tourist House - Sablino
15,Noyon,FR,81.25,overcast clouds,49.5833,3.0000,Cit'Hotel le Cèdre
20,Nara,JP,79.50,overcast clouds,34.6851,135.8049,Guesthouse Nara Komachi
21,Fort-Shevchenko,KZ,77.36,broken clouds,44.5086,50.2630,Oasis hotel Bautino
22,Hami,CN,78.66,clear sky,42.8000,93.4500,Laoyutang Resort
26,Karpathos,GR,76.96,clear sky,35.5070,27.2132,Alex
27,Jamestown,US,79.02,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
28,Hasaki,JP,79.29,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
30,Alexandria,EG,80.01,clear sky,31.2156,29.9553,Royal Crown Hotel


In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))